## Comment letters

Read the comment letters, compute doc2vec vector (for each letter) and group similar letters in 10 buckets

See https://towardsdatascience.com/automatic-topic-clustering-using-doc2vec-e1cea88449c
and https://github.com/olafmaas/hackdelft/blob/master/hackdelft/doc2vec/generate_json.py            

In [1]:
!pip install gensim

Defaulting to user installation because normal site-packages is not writeable


In [12]:
# init summary file
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.cluster import  hierarchy

with open(r'/blue/acg7849/share/BS/summary.txt') as f:
    files = [{k: v for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True, delimiter="|")]

In [13]:
# files with length > 1000
ffiles = [f for f in files if int(f["length"])> 1000 and (f["date"][0:4]) == '2019']

In [39]:
directory = (r'/blue/acg7849/share/BS/summary.txt')
directory = (r'/blue/acg7849/share/BS/item1/')

In [49]:
import gensim
import os, string, re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english') )

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

# add some punctuation to string.punctuation
punc = string.punctuation + '“”'

# documents get tagged by an index (number), while filenames have different numbers
# keep track of this
fileIdToIndex = {} # given a fileId -> tag
indexToFileId=[] # given a tag -> fileId

class MyCommentLetters(object):
    def __init__(self, dirname, tokens_only = False):
        self.dirname = dirname
        self.tokens_only = tokens_only
 
    def __iter__(self):
        for i, fname in ffiles:
            #files/directory/readBSGen do not work 
            #print(i, fname)
        #for fname in os.listdir(self.dirname):
            with open( os.path.join(self.dirname, fname), encoding='utf-8') as f:
                content = f.read()
            
            # grab id from filename
            myCounter = int (  re.findall(r'(\d*)\.txt', fname)[0] )
            # update 
            fileIdToIndex [ myCounter] = i
            indexToFileId.append( myCounter)
            #print('fname', fname, 'tag', myCounter)
            file_tokens = [x for x in word_tokenize(content) if x.isalpha() and x.lower() not in stopWords and x not in string.punctuation]
            
            if self.tokens_only == True:
                yield file_tokens
            else:
                yield TaggedDocument(words=file_tokens, tags=[i] )                    

In [50]:
# Hipergator
wordLists = MyCommentLetters(r'/blue/acg7849/share/BS/item1') # a memory-friendly iterator

In [51]:
# create a model, build vocabulary
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=40)
model.build_vocab(wordLists)

ValueError: too many values to unpack (expected 2)

In [ ]:
fileIdToIndex

In [ ]:
indexToFileId[ 50  ]

In [ ]:
myText = '10-K'
print( myText.isalpha() )

In [ ]:
# train it
model.train(wordLists, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
# Hipergator
def tokenizeFile(file_id):
    with open( r'/blue/acg7849/share/comment_letters/'+str(file_id)+'.txt', encoding='utf-8') as f:
            content = f.read()
    return ([x for x in word_tokenize(content) if x.isalpha() and x.lower() not in stopWords and x not in string.punctuation] )

In [ ]:
t = tokenizeFile(1)
model.infer_vector( t )

In [ ]:
t = tokenizeFile(1)
inferred_vector = model.infer_vector( t )
# dv is short for docvecs
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
#sims = model.dv.most_similar([inferred_vector], topn=4)
sims

In [ ]:
# letter with filename 1.txt is the first letter, so tag is 0
similar_doc = model.docvecs.most_similar(0)
similar_doc

In [ ]:
print('number of documents', model.corpus_count)
print('model.docvecs', len(model.dv))

In [ ]:
# Hipergator
# reread the files, and get the vector for each file
# feed vector into k-means algorithm to make clusters
wordLists = MyCommentLetters(r'/blue/acg7849/share/comment_letters/', tokens_only = True) # a memory-friendly iterator
vectors = [ model.infer_vector( w ) for w in wordLists]
len(vectors)

In [ ]:
vectors[0]

In [ ]:
import nltk
from nltk.cluster import KMeansClusterer
num_clusters = 10
kclusterer = KMeansClusterer(num_clusters, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(vectors, assign_clusters=True)

In [ ]:
assigned_clusters[0:20]

In [ ]:
import collections

print(collections.Counter(assigned_clusters))

## In-class question

For each cluster, what are the 20 most frequent words (excluding stop words, excluding punctuation)
